<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/114_%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E5%87%A6%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

テキスト処理
============

文字列の内部表現
----------------

Python の文字列の内部表現は、Unicode コードポイントの並びであり、UTF-8 や UTF-16 といった符号化方式を用いていない。メモリ効率最適化のため、含まれる文字の最大のコードポイントに応じて、以下のいずれかの形式が用いられる（[PEP 393](https://peps.python.org/pep-0393/)）。

  * **1-byte kind**（Latin-1）: 全ての文字が U+00FF 以下の場合。1 文字あたり 1 バイト。
  * **2-byte kind**（UCS-2）: 全ての文字が U+FFFF 以下（基本多言語面、BMP）の場合。1 文字あたり 2 バイト。
  * **4-byte kind**（UCS-4）: U+10000 以上の補助面の文字（絵文字など）を含む場合。1 文字あたり 4 バイト。

CPython の文字列の実装は構造体で、文字列データは `(文字数 + 1) * (kindに応じたバイト数)` の連続したメモリ領域に格納される。`+1` は C との互換性のための終端 NULL 文字の分である（[cpython/Objects/unicodeobject.c](https://github.com/python/cpython/blob/0d7f61ddb074659d8c18c8f5ac86a6a18e41f9e5/Objects/unicodeobject.c#L1447)）。

次のコードは、`sys.getsizeof()` 関数を使用して、文字列のメモリ消費量を比較する例である。`sys.getsizeof()` が返す値は構造体全体のメモリ消費量となるので、差分を取って 1 文字分のメモリ消費量を計算している。

In [ ]:
import sys
x = 'abc'
assert sys.getsizeof(x + 'a') - sys.getsizeof(x) == 1
x = 'abcあ'
assert sys.getsizeof(x + 'a') - sys.getsizeof(x) == 2
x = 'abc𠮷'
assert sys.getsizeof(x + 'a') - sys.getsizeof(x) == 4

組み込み関数 `ord()` は、1 文字の文字列を受け取り、その Unicode コードポイントを表す整数を返す。

組み込み関数 `chr(i)` は、`ord()` の逆の操作であり、整数を受け取り、その値を Unicode コードポイントとする 1 文字の文字列を返す。引数の有効な範囲は 0 から 1,114,111（ 16 進数で 0x10FFFF）。

In [ ]:
assert ord('a') == 97       # U+0061
assert ord('あ') == 12354   # U+3042
assert ord('𠮷') == 134071  # U+20BB7
assert chr(97) == 'a'
assert chr(12354) == 'あ'
assert chr(134071) == '𠮷'

文字列演算
----------

### 共通のシーケンス演算

Python の文字列は、イミュータブルな Unicode コードポイントのシーケンスとして扱われる。

Python の文字列は、共通のシーケンス演算をサポートする。インデックス（添字）を指定して文字や部分文字列を取得する方法については、[公式チュートリアル](https://docs.python.org/ja/3/tutorial/introduction.html#text)で十分。

In [ ]:
s = "spam ham "
t = "eggs"
x = "am"
n = 3
i = 5
j = 8
k = 2

# s のある要素が x と等しければ True , そうでなければ False
assert x in s

# s のある要素が x と等しければ False, そうでなければ True
assert not (x not in s)

# s と t の結合
assert s + t == "spam ham eggs"

# s 自身を n 回足すのと同じ
assert s * n == n * s == "spam ham spam ham spam ham "

# s の 0 から数えて i 番目の要素
assert s[i] == "h"

# s の i から j までのスライス
assert s[i:j] == "ham"
assert s[:] == s  # s のコピー

# s 中で x が最初に出現するインデックス (インデックス i 以降からインデックス j までの範囲)
assert s.index(x) == 2
assert s.index(x, i) == s.index(x, i, j) == 6

# s 中に x が出現する回数 (インデックス i 以降からインデックス j までの範囲)
assert s.count(x) == 2
assert s.count(x, i, j) == 1

文字列はイミュータブルなので、インデックス参照に対して代入で直接書き換えようとするとエラーとなる。

In [ ]:
word = "live"
try:
    word[1] = "o"
except TypeError as e:
    print(e)
# 文字を置き換えた新しい文字列を作成するならOK
word = word[:1] + "o" + word[2:]
assert word == "love"

'str' object does not support item assignment


`str` オブジェクトは Unicode コードポイントを保持し、内部表現形式（kind）を把握しているため、`len()` 関数はバイト数ではなく常に Unicode コードポイントの数を返す。UTF-16 を使用していないので、BMP 外の文字もサロゲートペアを考慮せずに扱える。

In [ ]:
assert len('abc') == 3
assert len( 'abcあ') == 4
assert len('abc😀') == 4  # BMP 外の文字でも普通にコードポイント数を返せる

通常、Unicode コードポイントの数は、見た目の文字数と一致する。ただし、結合文字やゼロ幅ジョインナー（ZWJ）を使った絵文字などは複数のコードポイントを組み合わせて 1 つの見た目の文字を構成するため、`len()` の結果が直感的な文字数と異なる場合がある。

  * 結合文字: `é` ＝ `e` U+0065（101） ＋ ` ́` U+0301（769） &emsp; ※ é は独自のコードポイント U+00E9（233）も持っている
  * ZWJ合成: `👨‍👩‍👧‍👦` ＝ `👨` U+1F468（128104） ＋ ZWJ U+200D（8205） ＋ `👩` U+1F469（128105） ＋ ZWJ U+200D（8205） ＋ `👧` U+1F467（128103） ＋ ZWJ U+200D（8205） ＋ `👦` U+1F466（128102）

In [ ]:
assert len('é') == 2  # 結合文字
assert len('👨‍👩‍👧‍👦') == 7  # ゼロ幅ジョインナーを使った絵文字

`max()`, `min()` 関数は、それぞれ Unicode コードポイントが最大、最小の文字を返す。

In [ ]:
assert ord('い') == 12356
assert ord('ろ') == 12429
assert ord('は') == 12399
assert max('いろは') == 'ろ'
assert min('いろは') == 'い'

文字列をイテレーターとして使用すると、Unicode コードポイントごとに文字が返される。

結合文字やゼロ幅ジョインナーを使った絵文字などは、見た目の文字を構成する複数のコードポイントに対応する文字が、それぞれ個別に返される。

In [ ]:
for c in "abcあ😀é👨‍👩‍👧‍👦":
    print(f"{c}: {ord(c)}")

a: 97
b: 98
c: 99
あ: 12354
😀: 128512
e: 101
́: 769
👨: 128104
‍: 8205
👩: 128105
‍: 8205
👧: 128103
‍: 8205
👦: 128102


### 比較演算

Python の文字列比較は、Unicode コードポイントの大小を比較の基準とし、数学でいう「辞書式順序」に従って判定される。

以下は代表的な文字のコードポイント範囲である。

| 区分 | 例 | Unicode コードポイント範囲 | 備考 |
|:---|:---|:---|:---|
| 半角数字 | `0` – `9` | U+0030 – U+0039 | ASCII 領域 |
| 半角英大文字 | `A` – `Z` | U+0041 – U+005A | ASCII 領域 |
| 半角英小文字 | `a` – `z` | U+0061 – U+007A | ASCII 領域 |
| ひらがな | `あ` – `ん` | U+3041 – U+3096 | 小書き文字もこの範囲に含まれる |
| カタカナ | `ア` – `ン` | U+30A1 – U+30FA | 小書き文字もこの範囲に含まれる |
| 長音記号 | `ー` | U+30FC | BMP（基本多言語面）領域 |
| 全角数字 | `０` – `９` | U+FF10 – U+FF19 | BMP（基本多言語面）領域 |
| 全角英大文字 | `Ａ` – `Ｚ` | U+FF21 – U+FF3A | BMP（基本多言語面）領域 |
| 全角英小文字 | `ａ` – `ｚ` | U+FF41 – U+FF5A |BMP（基本多言語面）領域 |
| 半角カタカナ | `ｱ` – `ﾝ` | U+FF67 – U+FF9D | BMP（基本多言語面）領域 |

**小書き文字**は「あ」に対する「ぁ」のように小字で表される仮名であり、一般に

$$
\text{小書き文字} < \text{対応する普通の仮名}
$$

である。例外は「ゕ」「ゖ」「ヵ」「ヶ」で、それぞれ「か」「け」「カ」「ケ」よりも大きい。

また、普通の仮名に対して、濁点「゛」付きの場合と半濁点「゜」付きの場合の大小は次のとおり。

$$
\text{普通の仮名} < \text{濁点「゛」付きの仮名} < \text{半濁点「゜」付きの仮名}
$$

2 つの文字列を比較するアルゴリズム（辞書式順序）は以下の通り:

  1. 両方の文字列の最初の文字を比較する。
  2. 1 つ目のほうが大きい（もしくは小さい）場合、1 つ目の文字列は 2 つ目の文字列よりも大きい（もしくは小さい）。それで完了。
  3. そうではなく、もしも最初の文字が等しい場合は、同じ方法で 2 つ目の文字を比較する。
  4. 文字列の最後までそれを繰り返す。
  5. 両方の文字列が同時に終わった場合、それらは等しい。そうでなければ長い文字列がより大きい。

In [ ]:
assert '0' < 'A' < 'a' < 'あ' < 'ア' < '０' < 'Ａ' < 'ａ' < 'ｱ'
assert '自由が丘' < '自由ヶ丘'
assert 'ハンドル' < 'バンド' < 'パン'
assert 'スタンプ' < 'スター'

発音区別記号付きの文字を持つ言語では、Unicode コードポイントの大小がアルファベット順に反している。

In [ ]:
assert 'Österreich' > 'Zealand'

`locale` モジュールを使うことで、ロケールに依存した文字列大小比較が可能となる。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `locale.strcoll(string1, string2)` | 現在の `locale.LC_COLLATE` 設定に従って 2 つの文字列を比較する。`string1` が `string2` に対して前<br /><br />に来るか、後に来るか、あるいは 2 つが等しいかによって、それぞれ負の値、正の値、あるいは 0 を返す | `int` |
| `locale.strxfrm(string)` | 文字列を、ロケールを考慮した比較に使える形式に変換する。例えば、`strxfrm(s1) < strxfrm(s2)` は<br /><br /> `strcoll(s1, s2) < 0` と等価 | `str` |

``` python
>>> import locale
>>> locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')
'de_DE.UTF-8'
>>> locale.strcoll('Österreich', 'Zealand')
-1
>>> locale.strxfrm('Österreich') < locale.strxfrm('Zealand')
True
>>> locale.strcoll('straße', 'strasse')
0
>>> locale.strxfrm('straße') == locale.strxfrm('strasse')
True
>>>
```

この例では、ドイツ語ロケールでは「ß」と「ss」が同等と扱われるため、通常の `==` では異なる文字列でも `locale.strcoll()` では同等として評価される。

`locale.strxfrm()` 関数は、ソート関数に比較キーを与えるのに使うことができる。

``` python
>>> import locale
>>> locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')
'de_DE.UTF-8'
>>> words = ['Zealand', 'Österreich']
>>> sorted(words, key=locale.strxfrm)
['Österreich', 'Zealand']
>>>
```

文字列メソッド
--------------

### 文字列チェック

文字列オブジェクトが持つ次のメソッドは、文字列が特定の文字を含むかどうかをチェックするもので、戻り値はすべて bool 型である。

| メソッド | 機能 |
|:--|:--|
| `isalpha()` | 文字列が（日本語などの非 ASCII 文字が含まれていても）数字と記号を含まず、かつ 1 文字以上あるなら `True` を返す。 |
| `isascii()` | 文字列が空であるか、文字列の全ての文字が ASCII（U+0000-U+007F）である場合に `True` を返す |
| `isdecimal()` | 文字列中の全ての文字が十進数字で、かつ 1 文字以上あるなら `True` を返す。十進数字に全角数字も含まれるが、漢数字は含まれない |
| `isdigit()` | 文字列中の全ての文字が数字で、かつ 1 文字以上あるなら `True` を返す。数字に全角数字が含まれるほか、①❶⓵などの丸数字、¹₁などの<br /><br />上付き数字・下付き数字、⒈⑴なども含まれるが、漢数字は含まれない |
| `isnumeric()` | 文字列中の全ての文字が数を表す文字で、かつ 1 文字以上あるなら `True` を返す。数を表す文字に `isdigit()` で真となる文字が含まれ<br /><br />るほか、漢数字やⅠⅰなどの全角ローマ数字なども含まれる |
| `isalnum()` | `str.isalpha() or str.isdecimal() or str.isdigit() or str.isnumeric()` と同じ |
| `isidentifier()` | 識別子として使用できる文字列（つまり Python の変数名に使える文字列）なら `True` を返す |
| `islower()` | 文字列が全て小文字で、かつ 1 文字以上あるなら `True` |
| `isupper()` | 文字列が全て大文字で、かつ 1 文字以上あるなら `True` |
| `istitle()` | 文字列中の単語の先頭のみ大文字であとは小文字で、かつ 1 文字以上あるなら `True` |
| `isprintable()` | 文字列が全て印字可能文字であるか、文字列が空であれば `True`。改行や復帰、タブなどの制御文字は印字不可能文字 |
| `isspace()` | 文字列が空白だけからなり、かつ 1 文字以上ある場合には `True` を返す。空白にタブと改行、全角スペースも含まれる |

In [ ]:
text = "abcあいうえお日本語"
assert (text.isalpha(), text.isalnum()) == (True, True)  # 非 ASCII 文字でも数字と記号でなければ True
assert not (text + "#").isalnum()
num = "0123456789"
assert (num.isdecimal(), num.isdigit(), num.isnumeric()) == (True, True, True)
num = "０１２３４５６７８９"
assert (num.isdecimal(), num.isdigit(), num.isnumeric()) == (True, True, True)
num = "⓪①②➂④⑤⑥⑦⑧⑨"
assert (num.isdecimal(), num.isdigit(), num.isnumeric()) == (False, True, True)
num = "ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ"
assert (num.isdecimal(), num.isdigit(), num.isnumeric()) == (False, False, True)
num = "壱十百千万億兆"  # 京・垓は isnumeric() でも False を返す
assert (num.isdecimal(), num.isdigit(), num.isnumeric()) == (False, False, True)
assert not "spam-ham".isidentifier()  # ハイフン '-' は識別子に使えない
assert "ABCÀÆÇÈ×ØÜ".isupper()
assert "abcàæçè÷øü".islower()
assert "Title Case".istitle()
assert " \n　".isspace()

### 文字列変換

文字列オブジェクトが持つ次のメソッドは、文字列を変換するもので、戻り値はすべて `str` 型である。文字列オブジェクト自体は変更不可能（イミュータブル）なので変更されないことに注意する。

| メソッド | 機能 |
|:--|:--|
| `lower()` | 文字列をすべて小文字に変換した文字列を返す |
| `upper()` | 文字列をすべて大文字に変換した文字列を返す |
| `swapcase()` | 大文字を小文字に、小文字を大文字に変換した文字列を返す |
| `capitalize()` | 最初の文字を大文字にし、残りを小文字にした文字列を返す |
| `title()` | 単語ごとに大文字から始まり、残りを小文字にした文字列を返す |
| `replace(old, new[, count])` | 部分文字列 old を全て new に置換した文字列を返す。オプション引数 count が与えられている場合、先頭から count 個の old <br /><br />だけを置換する。Python 3.13 以降では、`count` はキーワード引数として指定可能 |
| `strip([chars])` | 文字列の先頭および末尾から指定した文字集合 chars に含まれる文字をすべて除去した文字列を返す。chars が省略されるか<br /><br /> None の場合、空白文字（改行文字とタブ文字、全角スペースを含む）が除去される |
| `lstrip([chars])` | 文字列の先頭から指定した文字集合 chars に含まれる文字をすべて除去した文字列を返す。chars が省略されるか None の場<br /><br />合、空白文字（改行文字とタブ文字、全角スペースを含む）が除去される |
| `rstrip([chars])` | 文字列の末尾から指定した文字集合 chars に含まれる文字をすべて除去した文字列を返す。chars が省略されるか None の場<br /><br />合、空白文字（改行文字とタブ文字、全角スペースを含む）が除去される |
| `removeprefix(prefix, /)` | 文字列の先頭から prefix で指定した文字列を除去した文字列を返す |
| `removesuffix(suffix, /)` | 文字列の末尾から suffix で指定した文字列を除去した文字列を返す |
| `center(width[, fillchar])` | width の長さをもつ中央寄せされた文字列を返す。fillchar で埋める文字を指定する（デフォルトは ASCII スペース）。width が文<br /><br />字列の長さ以下なら元の文字列を返す |
| `ljust(width[, fillchar])` | width の長さをもつ左揃えした文字列を返す。fillchar で埋める文字を指定する（デフォルトは ASCII スペース）。width が文字列<br /><br />の長さ以下なら元の文字列を返す |
| `rjust(width[, fillchar])` | width の長さをもつ右揃えした文字列を返す。fillchar で埋める文字を指定する（デフォルトは ASCII スペース）。width が文字列<br /><br />の長さ以下なら元の文字列を返す |
| `expandtabs(tabsize=8)` | タブ文字による位置調整を、tabsize をタブ幅とする ASCII スペースで位置調整した文字列を返す |
| `zfill(width)` | 長さが width になるよう 0 で左詰めした文字列を返す。先頭が +/- なら符号の後に 0 を挿入する。符号や小数点も含めた長さ<br /><br />を width にする |

In [ ]:
text = "HELLO world!"
assert text.upper() == "HELLO WORLD!"
assert text.lower() == "hello world!"
assert text.swapcase() == "hello WORLD!"
assert text.capitalize() == "Hello world!"
assert text.title() == "Hello World!"
assert text.replace("world", "python") == "HELLO python!"
assert text.replace("L", "l", 1) == "HElLO world!"

text = " あいうえお日本語 "
assert text.lstrip() == "あいうえお日本語 "
assert text.lstrip("あ 語日本") == "いうえお日本語 "  # 左からは 'い' で文字集合に含まれない文字が現れるので、その直前までを除去
assert text.rstrip() == " あいうえお日本語"
assert text.rstrip("あ 語日本") == " あいうえお"  # 右からは 'お' で文字集合に含まれない文字が現れるので、その後ろまでを除去
assert text.strip() == "あいうえお日本語"
assert text.strip("あ 語日本") == "いうえお"
assert text.removeprefix(" あいうえお") == "日本語 "
assert text.removesuffix("日本語 ") == " あいうえお"

assert "中央寄せ".center(10) == "   中央寄せ   "
assert "中央寄せ".center(10, "#") == "###中央寄せ###"
assert "左揃え".ljust(10) == "左揃え       "
assert "左揃え".ljust(10, "#") == "左揃え#######"
assert "右揃え".rjust(10) == "       右揃え"
assert "右揃え".rjust(10, "#") == "#######右揃え"

assert "a\tbcdefg\th".expandtabs() == "a       bcdefg  h"
assert "12".zfill(10) == "0000000012"
assert "-1.2".zfill(10) == "-0000001.2"

`str` は、変換テーブルを使用する文字列変換もサポートしている。以下のメソッドを使用する。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `str.maketrans(x[, y[, z]])` | スタティックメソッド。`translate()` メソッドに使える変換テーブルを返す | `dict` |
| `translate(table)` | 変換テーブルを使って複数の文字を置換した新しい文字列を返す | `str` |

変換テーブルは、マッピングオブジェクトで、キーが Unicode コードポイントに対応する 10 進整数、値が長さ 1 以上の文字列であるマッピングオブジェクトまたは `None` である必要がある。文字列オブジェクトの `translate()` メソッドに変換テーブルを渡すと、文字列の中のキーに該当する文字を、キーに対応する値に全て置き換えた文字列が返される。キーに対応する値が `None` なら、該当する文字が削除された文字列が返される。

変換テーブルを直接定義することも可能だが、 `str.maketrans()` スタティックメソッドを使うほうが簡単である。

`str.maketrans()` に引数を 1 つだけ与える場合、辞書を渡す必要がある。その辞書はほとんどそのまま変換テーブルになるのであるが、キーを長さ 1 の文字列とすることができ、`str.maketrans()` により Unicode コードポイントに対応する 10 進整数に変換される。

`str.maketrans()` に引数を 2 つ指定する場合、長さが同じ文字列を 2 つ渡す必要がある。第 1 引数の各文字がそれぞれ第 2 引数の同じ位置の文字に対応づけされた変換テーブルが作成される。第 3 引数を指定する場合、文字列を指定する必要があり、それに含まれる文字が全て `None` に対応づけられる。

In [ ]:
table = str.maketrans({"A": "Aaaa", "p": "P", "!": None})
print(f"{table=}")
assert "Apple!".translate(table) == "AaaaPPle"

table = str.maketrans("ー０１２３４５６７８９", "-0123456789")
assert "０１２－345-６７８９".translate(table) == "012－345-6789"

table={65: 'Aaaa', 112: 'P', 33: None}


### 探索・分割・結合

文字列オブジェクトが持つ次のメソッドは、文字列を探索・分割・結合するものである。文字列オブジェクト自体は変更不可能（イミュータブル）なので変更されないことに注意する。

| メソッド | 機能 | 戻り値 |
|:--|:--|:-:|
| `find(sub[, start[, end]])` | 文字列中に部分列 `sub` が最初に出現する位置（位置 `start` 以降から位置 `end` までの範囲）を返す。存在<br /><br />しなければ -1 を返す<br /><br />※ sub が部分列であるかどうかのみを調べるには、`in` 演算子を使うべきである | `int` |
| `rfind(sub[, start[, end]])` | 文字列中に部分列 `sub` が含まれる場合、最も右にあるものの位置（位置 `start` 以降から位置 `end` までの<br /><br />範囲）を返す。存在しなければ -1 を返す | `int` |
| `rindex(sub[, start[, end]])` | `rfind()` と同様であるが、`sub` が見つからなかった場合 `ValueError` 例外を送出する | `int` |
| `startswith(prefix[, start[, end]])` | 文字列が指定された `prefix` で始まる（位置 start 以降から位置 end までの範囲）なら `True` を返す。<br /><br />`prefix` にはタプルで複数の候補を指定できる | `bool` |
| `endswith(suffix[, start[, end]])` | 文字列が指定された `suffix` で終わる（位置 start 以降から位置 end までの範囲）なら `True` を返す。<br /><br />`suffix` にはタプルで複数の候補を指定できる | `bool` |
| `split(sep=None, maxsplit=-1)` | 文字列を区切り文字列 `sep` で区切った単語のリストを返す。`maxsplit` が与えられていれば、最大で<br /><br /> `maxsplit` 回分割される（デフォルトの -1 なら、回数制限なし）。`sep` が `None` 以外で与えられた場合と、そ<br /><br />うでない場合では、分割アルゴリズムが異なる<br /><br />・`sep` が `None` 以外で与えられた場合: 連続した区切り文字はまとめられず、空の文字列を区切っていると<br /><br />　判断される<br /><br />・`sep` が与えられないか `None` の場合: 空白文字（タブ文字と全角スペースを含む）で区切る。連続する空白<br /><br />　文字は 1 つの区切りとみなされる。また、文字列の先頭や末尾に空白があっても、結果の最初や最後に空<br /><br />　文字列は含まれない | `list` |
| `rsplit(sep=None, maxsplit=-1)` | `maxsplit` が与えられた場合、文字列の右端から最大 `maxsplit` 回分割を行うこと以外は、`split()` と同<br /><br />様に振る舞う | `list` |
| `splitlines(keepends=False)` | 文字列を改行部分で分解し、各行からなるリストを返す。`keepends` に真が与えらない限り、改行をリストに<br /><br />含めない | `list` |
| `partition(sep)` | 文字列を区切り文字列 `sep` の最初の出現位置で区切り、区切りの前の部分、区切り文字列そのもの、区切<br /><br />りの後ろの部分という 3 要素のタプルを返す。もし区切れなければ、タプルには元の文字列そのものとその<br /><br />後ろに二つの空文字列が入る | `tuple` |
| `rpartition(sep)` | 文字列を区切り文字列 `sep` の最後の出現位置で区切り、区切りの前の部分、区切り文字列そのもの、区切<br /><br />りの後ろの部分という 3 要素のタプルを返す。もし区切れなければ、タプルには二つの空文字列とその後ろ<br /><br />に元の文字列そのものが入る | `tuple` |
| `join(iterable)` | 文字列を区切り文字列として、`iterable` 中の文字列を結合した文字列を返す。文字列が空であれば、<br /><br />iterable 中の文字列を単純に結合した文字列を返す。`iterable` に bytes オブジェクトのような非文字列の<br /><br />値が存在するなら、TypeError を送出する | `str` |

**エディタの入力補完では、 `*strip()` と `*split()` の入力ミスに気付かずプログラムが意図しない動作をするということがあるので十分に注意する**。

In [ ]:
assert "python".find("th") == 2
assert "python".find("th", 3) == -1
assert "Beautiful is better than ugly.".rfind("t") == 20
assert "python".startswith("py")
assert "image.png".endswith(("jpg", "png", "gif"))
assert " peek  a  boo\ni  see  you ".split() == ["peek", "a", "boo", "i", "see", "you"]
assert "peek a boo\ni see you".split("e") == ["p", "", "k a boo\ni s", "", " you"]
assert "peek a boo\ni see you".split("u") == ["peek a boo\ni see yo", ""]
assert "peek a boo\ni see you".rsplit(maxsplit=3) == ["peek a boo", "i", "see", "you"]
assert "peek a boo\ni see you".splitlines() == ["peek a boo", "i see you"]
assert "Simple is better than complex.".partition("pl") == ("Sim", "pl", "e is better than complex.")
assert "Simple is better than complex.".rpartition("pl") == ("Simple is better than com", "pl", "ex.")
assert "-".join("python") == "p-y-t-h-o-n"
assert "".join(["py", "thon"]) == "python"

文字列とバイト列
----------------

### エンコードとデコード

テキストデータを含めてコンピュータが扱うデータは全てバイナリデータであり、2 進数の組み合わせに意味付けをしたものである。テキストデータは、文字や記号を一定の規則に従って、2 進数の組み合わせに対応付けたものである。

Python はテキストデータを `str` 型（文字列）として、内部的には Unicode コードポイントの並びを保持しているのであるが、外部では通常、一定の**符号化方式**（encoding）によってテキストデータが保存・伝送される。そのため、Python の文字列と外部の符号化済みデータを相互にやり取りする際には、どの符号化方式を用いるかを明示する必要がある。

例えば、ソースコードの文字コード形式を UTF-8 以外とする場合には、エンコーディング宣言を記述する必要がある（PEP 8 では UTF-8 以外の利用は非推奨）。エンコーディング宣言は、Python スクリプトの 1 行目または 2 行目に `'coding'` と文字コード形式を含むコメントとして書かれる。次のような宣言は GNU Emacs をはじめとする多くのテキストエディタで認識される。

``` python
# -*- coding: 文字コード形式 -*-
```

Python で指定できる文字コード形式は、公式ドキュメントの[標準エンコーディング](https://docs.python.org/ja/3/library/codecs.html#standard-encodings) と[テキストエンコーディング](https://docs.python.org/ja/3/library/codecs.html#text-encodings)を参照。以下が代表的。

| 文字コード形式 | 別名 | 言語 | 注意事項 |
|:---|:---|:---|:---|
| `ascii` | `646`, `us-ascii` | 英語 | |
| `utf_8` | `U8`, `UTF`, `utf8`, `cp65001` | 全ての言語 | BOM なし（Windows では UTF-8N と表記される） |
| `utf_8_sig` | |  全ての言語 | BOM 付き（Excel が CSV 形式で使用） |
| `latin_1` | `iso-8859-1`, `iso8859-1`,<br /><br /> `8859`, `cp819`, `latin`,<br /><br /> `latin1`, `L1` | 西ヨーロッパ言語 | |
| `shift_jis` | `csshiftjis`, `shiftjis`,<br /><br /> `sjis`, `s_jis` | 日本語 | |
| `cp932` | `932`, `ms932`, `mskanji`,<br /><br /> `ms-kanji` | 日本語 | Shift-JIS の拡張版で日本語版 Windows のデフォルトエンコーディング |
| `punycode` | | 全ての言語 | 国際化ドメイン名（IDNA）を既存の DNS で運用できるようにするために Unicode 文字を<br /><br />アルファベット、数字、ハイフン `-` の組み合わせに変換する符号化方式の 1 つ |

文字コード形式の指定では、大文字と小文字、アンダースコア `_` とハイフン `-` は区別されない。例えば `'utf-8'` は `'utf_8'` と同じである。

一方、Python は、バイナリデータをバイト、すなわち符号なし 8 ビット整数値（0～255）の列としてそのまま保持することもできる。これが `bytes` 型（バイト列）である。`bytes` はバイトの不変なシーケンスであるが、可変なバージョンとして `bytearray` 型も用意されている。

**バイト列リテラル**では、バイトを ASCII 文字コードポイントに対応付けて、ASCII 印字可能文字を直接書くことができる。その際、先頭に `b` または `B` を付けて引用符で囲む。引用符の種類は文字列リテラルと同じものが使える。また、エスケープシーケンスも文字列とほぼ同様に利用できるが、Unicode エスケープ（`\uxxxx`、`\Uxxxxxxxx`、`\N{name}`）は使用できない。さらに、8 進数形式の `\ooo` エスケープは最大 `\377`（=255）までしか表せない。

ASCII 制御文字や、ASCII に含まれない 128～255 のバイト値を表す場合には、適切なエスケープシーケンスを用いる必要がある。1 バイトの値はちょうど 16 進数 2 桁で表せるため、一般的には `\xhh` 形式が使われる。例えば、`0` は ASCII でヌル文字（null）に対応するので `b'\x00'` と書き、`128` は ASCII には含まれないがバイト値として `b'\x80'` と書ける。

先頭に `br` または `rb`（大文字小文字の組み合わせは自由）を付けると、raw 文字列リテラルのようにバックスラッシュ `\` がエスケープとして解釈されなくなる。

`str` と `bytes` は全く異なる型として扱われる。このため、`'abc'+b'abc'` のようにして文字列とバイト列を連結できない。文字列とバイト列は比較することもできず、たとえリテラルが同じ ASCII 文字であっても、あるいは互いに空であっても等価性は認められない。

In [ ]:
assert 'A' != b'A'
assert '' != b''

``` python
str.encode(encoding='utf-8', errors='strict')
```

文字列の `encode()` メソッドは、内部の Unicode コードポイント列を文字コード形式 `encoding` で変換したバイト列を返す（**エンコード**）。Python 文字列を外部ファイル保存や通信で使う場合、エンコードする必要がある。

変換できない文字があった場合、`errors` が `'strict'`（デフォルト）なら `UnicodeError` 例外が送出され、`'ignore'` ならその文字が無視され、`'replace'` なら `b'?'` に変換される。

`encode()` メソッドでは、特殊な文字コード形式として `'unicode_escape'` を指定できる。これを指定すると、ASCII 文字はそのまま、非 ASCII 文字は Unicode エスケープシーケンス形式の表現に変換されたバイト列が返される。これは、文字列が内部的にどのようなコードポイントで保持されているかを可読な形で確認したい場合などに便利である。

In [ ]:
assert b'A' == b'\x41' == 'A'.encode()
assert b'\xe3\x81\x82' == 'あ'.encode()
assert 'あ'.encode('ascii', 'ignore') == b''  # errors 引数が 'ignore' の場合、変換できない文字は b'' に置換される
assert 'あ'.encode('ascii', 'replace') == b'?'  # errors 引数が 'replace' の場合、変換できない文字は b'?' に置換される
assert 'あ'.encode('unicode_escape') == b'\\u3042'  # '\u3042' を表現するため、先頭の\をエスケープしたバイト列が返される
assert '\u3042' == 'あ'

``` python
bytes.decode(encoding='utf-8', errors='strict')
```

バイト列の `decode()` メソッドは、文字コード形式 `encoding` で符号化済みのバイト列から Python の文字列を返す（**デコード**）。外部ファイルや通信から取得したテキストデータを Python 文字列として扱うには、内部表現である Unicode コードポイント列に変換するデコードが必要である。

`str.encode()` とは逆の変換となり、`errors` 引数のエラー処理の種類は同様となる（ただし、`'replace'` のときの置換文字は異なり `�`（U+FFFD）になる）。

In [ ]:
assert b'\xe3\x81\x82'.decode() == 'あ'
assert b'\xe3\x81\x82'.decode('ascii', 'ignore') == ''
assert b'\xe3\x81\x82'.decode('ascii', 'replace') == '���'

デコードは、`str()` 関数でも可能である。

``` python
str(object=b'', encoding='utf-8', errors='strict')
```

`str()` は、`encoding` または `errors` 引数が指定された場合に限り、第 1 引数のバイト列を Python 文字列に変換する。

`str(bytes, encoding, errors)` は、`bytes.decode(encoding, errors)` と等価である。

In [ ]:
assert str(b'\xe3\x81\x82', 'utf-8') == 'あ'
assert str(b'\xe3\x81\x82', 'ascii', 'ignore') == ''
assert str(b'\xe3\x81\x82', 'ascii', 'replace') == '���'
# str()はバイト列だけを与えられた場合、そのリテラル表現の文字列を返す
assert str(b'\xe3\x81\x82') == "b'\\xe3\\x81\\x82'"

### bytes と bytearray の性質

バイト列をコンストラクタで作成する場合、引数は次のようになる。

``` python
bytes([source[, encoding[, errors]]])
```

  * 単一の引数として範囲 `0 <= x < 256` の整数からなるイテラブルを与える場合、それによって初期化されたバイト列を返す。
  * 単一の引数として正の整数を与える場合、その長さの null バイト（`b'\x00'`）のバイト列を返す。
  * 引数を与えない場合、空のバイト列 `b''` を返す。
  * 第 1 引数に文字列を与える場合、第 2 引数 `encoding` として文字コードを与えなければならない。この場合、オプションとして第 3 引数 `errors` も指定可能。`s.encode(encoding, errors)` は `bytes(s, encoding, errors)` と等価である。

In [ ]:
assert bytes([65]) == b'A'
assert bytes([227, 129, 130]) == b'\xe3\x81\x82' == bytes('あ', 'utf-8') == 'あ'.encode('utf-8')
assert bytes(5) == b'\x00\x00\x00\x00\x00'
assert bytes() == b''

バイト列をイテラブルとして扱う場合、1 バイトずつ、その整数値が返される。

In [ ]:
assert list(b'A') == [65]
assert list(b'Python') == [80, 121, 116, 104, 111, 110]
assert list(b'\xe3\x81\x82') == [227, 129, 130]

多くの文字列メソッドと同様のバイト列メソッドがサポートされている。文字列にあってバイト列にないメソッドは、`isdecimal()`、`isnumeric()`、`isidentifier()`、`isprintable()`、`encode()` しかない。一方、`decode()` メソッドおよび次のバイト列メソッドは、文字列には存在しない。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `bytes.fromhex(string)` | クラスメソッド。2 桁の 16 進数で書かれた文字列をバイト列に変換する。空白文字は無視される | `bytes` |
| `bytes.hex([sep[, bytes_per_sep]])` | `bytes.fromhex()` の逆変換。1 バイトを 2 桁の 16 進数で表現した文字列を返す | `str` |

In [ ]:
assert b'python'.replace(b'p', b'P') == b'python'.capitalize() == b'Python'
assert bytes.fromhex('41') == b'A'
assert b'A'.hex() == '41'

バイト列は不変だが、`bytearray` オブジェクトは可変で、 `bytes` オブジェクトと共通の操作に加えて、`sort()` メソッドを除いてリストと同様の操作もサポートしている。 `bytearray` に専用のリテラル構文はないので、コンストラクタを使って作成する。

In [ ]:
x = bytearray(b'abc')
assert list(x) == [97, 98, 99]
x[0] = 98
assert bytes(x) == b'bbc'

### struct

標準ライブラリの `struct` モジュールは、Python の値を書式文字列に従ってバイト列に変換する関数、および、その逆方向の変換をする関数を提供する。

| 関数 | 機能 | 戻り値 |
|:---|:---|:---|
| `struct.pack(format, v1, v2, ...)` | 書式文字列 `format` に従い値 `v1`, `v2`, ... をまとめ 1 つのバイト列を返す。引数は指定した `format` が要求<br /><br />する型と正確に一致することが必要 | `bytes` |
| `struct.unpack(format, buffer)` | バイト列 `buffer` を `pack(format, ...)` の戻り値であろうときの  `v1`, `v2`, ... をタプルで返す | `tuple` |

書式文字列は以下の形式とする:

``` python
[バイトオーダー][繰り返し回数]型コード
```

バイトオーダーに指定できる文字は、`'<'`, `'>'`, `'!'`, `'='`, `'@'` のいずれかである。`'<'` はリトルエンディアン、 `'>'` はビッグエンディアン、 `'!'` はネットワークのバイトオーダー（ビッグエンディアン）、 `'='` と `'@'` は CPU のネイティブ形式（Intel x86、AMD64 (x86-64) および Apple M1 はリトルエンディアン）を指定する。省略した場合、`'@'` であるとみなされる。 `'@'` はデータアライメントも CPU のネイティブ形式となり効率的。

型コードは以下のとおり。

| `typecode` | C の型 | Python の型 | バイト |
|:--:|:---|:---|:--:|
| `'b'` | signed char | `int` | 1 |
| `'B'` | unsigned char | `int` | 1 |
| `'i'` | signed int | `int` | 2 |
| `'I'` | unsigned int | `int` | 2 |
| `'l'` | signed long | `int` | 4 |
| `'L'` | unsigned long | `int` | 4 |
| `'q'` | signed long long | `int` | 8 |
| `'Q'` | unsigned long long | `int` | 8 |
| `'f'` | float | `float` | 4 |
| `'d'` | double | `float` | 8 |
| `'?'` | _Bool | `bool` | 1 |
| `'c'` | char | `bytes` | 1 |
| `'s'` | char[] | `bytes` | - |
| `'P'` | void* | `int` | - |

型コードの前に整数をつけ、繰り返し回数を指定することができる。たとえば、書式文字列 `'4h'` は `'hhhh'` と全く同じ意味である。ただし、型コードが `'s'` の場合、整数はバイトの長さとして解釈され、繰り返し回数としては解釈されない。たとえば、`'10s'` は単一の 10 バイト文字列が単一の Python バイト列に変換されることを意味する。

In [ ]:
from struct import pack, unpack
vals = (b'ABC', 123, 123.456)
b = pack('10sid', *vals)
assert b == b'ABC\x00\x00\x00\x00\x00\x00\x00\x00\x00{\x00\x00\x00w\xbe\x9f\x1a/\xdd^@'
assert unpack('10sid', b) == (b'ABC\x00\x00\x00\x00\x00\x00\x00', 123, 123.456)

文字列定数・テンプレート文字列・ヘルパー関数
--------------------------------------------

標準ライブラリの `string` モジュールは、一般的な文字列操作に便利な定数やクラス、関数などを提供している。

### 文字列定数

| 定数名 | 意味 |
|:--|:--|
| `string.ascii_lowercase` | 英小文字 'abcdefghijklmnopqrstuvwxyz' |
| `string.ascii_uppercase` | 英大文字 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' |
| `string.ascii_letters` | `ascii_lowercase` と `ascii_uppercase` を合わせたもの |
| `string.digits` | 10 進数の数字 '0123456789' |
| `string.hexdigits` | 16 進数の数字 '0123456789abcdefABCDEF' |
| `string.octdigits` | 8 進数の数字 '01234567' |
| `string.punctuation` | 記号の文字列 '!"#$%&'()*+,-./:;<=>?@[\]^_`{&#124;}~' |
| `string.whitespace` | 空白として扱われる ASCII 文字 ' \t\n\r\x0b\x0c' |
| `string.printabl` | `digits`、`ascii_letters`、`punctuation` および `whitespace` を組み合わせたもの |

### テンプレート文字列

Python の**テンプレート文字列**（template strings）は、`$` に基づいた置換をサポートしていて、次の規則が使われている:

  1. `$$` はエスケープ文字。 `$` 一つに置換される。
  2. `$名前` は、名前をキーとする辞書の値で置換されるプレースホルダーを指定する。デフォルトでは、名前は大文字と小文字を区別しない ASCII 英数字（アンダースコアを含む）からなる文字列に制限される。文字列はアンダースコアか ASCII 文字から始まるものでなければならない。`$` の後に名前に使えない文字が出現すると、そこでプレースホルダ名の指定が終わる。
  3. `${名前}` は `$名前` と同じ。プレースホルダー名の後ろに名前として使える文字列が続いていて、それをプレースホルダー名の一部として扱いたくない場合、例えば `"${noun}ification"` のような場合に必要な書き方となる。

`string` モジュールでは、上記のような規則を実装したクラスを提供している。

``` python
string.Template(template)
```

コンストラクタはテンプレート文字列になる引数を 1 つだけ取る。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `substitute(mapping={}, /, **kwds)` | テンプレート置換を行い、新たな文字列を生成して返す。`mapping` はテンプレート中のプレースホルダー<br /><br />に対応するキーを持つような任意の辞書とする。辞書を指定する代わりに、キーワード引数も指定でき、<br /><br />その場合にはキーワードをプレースホルダー名に対応させる。`mapping` と `kwds` の両方が指定され、内<br /><br />容が重複した場合には、`kwds` に指定したプレースホルダを優先する | `str` |
| `safe_substitute(mapping={}, /, **kwds)` | `substitute()` と同じだが、例外発生時の処理が異なる | `str` |

`substitute()` は、プレースホルダーに対応するものを `mapping` 引数や `kwds` 引数から見つけられなかった場合に、`KeyError` 例外を送出する。一方、`safe_substitute()` は、 `KeyError` 例外を送出する代わりにもとのプレースホルダーがそのまま入る。

また、`substitute()` は、規則以外の書き方で文字列中に `$` を使った場合に `ValueError` 例外を送出する。一方、`safe_substitute()` は、 `ValueError` 例外を送出せず単に `$` を返す。

次のコードは、`Template` の使用例である:

In [ ]:
from string import Template
d = {"who": "Tim", "what": "shusi"}
s = Template("$who likes $what")
assert s.safe_substitute(d) == "Tim likes shusi"

### ヘルパー関数

`string` モジュールは、次のヘルパー関数を提供する。

``` python
string.capwords(s, sep=None)
```

この関数は、`str.split()` を使って引数を単語に分割し、 `str.capitalize()` を使ってそれぞれの単語の先頭の文字を大文字に変換し、 `str.join()` を使ってつなぎ合わせる。オプションの第 2 引数 `sep` が与えられないか `None` の場合、この置換処理は文字列中の連続する空白文字をスペース 1 つに置き換え、先頭と末尾の空白を削除する。それ以外の場合には `sep` は `str.split()` と `str.join()` に使われる。

In [ ]:
import string
sentence = "  Python  is one of the best programming languages."
assert string.capwords(sentence) == "Python Is One Of The Best Programming Languages."
assert sentence.title() == "  Python  Is One Of The Best Programming Languages."  # title() は区切り文字の処理をしない

正規表現
--------

### 正規表現の構文

**正規表現**（regular expression）は、あるパターンの文字列を表現する文字列である。正規表現の中で文字列を表現するために使用される特殊文字を**メタ文字**と呼ぶ。

メタ文字:

| メタ文字 | 意味 |
|:--:|:---|
| `[]` | 集合の中の 1 文字。連続した文字の範囲を `-` を 2 つの文字で挟んで指定できる |
| `[^]` | 集合に含まれていない 1 文字。連続した文字の範囲を `-` を 2 つの文字で挟んで指定できる |
| `. ` | 改行以外の 1 文字 |
| `^` | 文字列の先頭 |
| `$` | 文字列の末尾 |
| `*` | 直前の文字の 0 回以上の繰り返し |
| `+` | 直前の文字の 1 回以上の繰り返し |
| `?` | 直前の文字の 0 回か 1 回の繰り返し |
| `{n}` | 直前の文字の n 回の繰り返し |
| `{m, n}` | 直前の文字の m 回以上 n 回以下の繰り返し |
| `A`&#124;`B` | `A` か `B` のどちらか |
| `()` | グループの開始と終了 |
| `(?P<name>...)` | 名前 `name` が付いたグループの開始と終了。`name` は有効な Python 識別子である必要がある |

※ メタ文字 `[]^.$*+?{}()<>|` を普通の文字としてマッチさせたいときは、`\[` のようにエスケープする必要がある。`\` 自体をマッチさせたいときは、`\\` とエスケープする必要がある。

正規表現で使う特殊なエスケープシーケンス:

| 特殊シーケンス | 意味 |
|:--:|:---|
| `\d` | `[0-9]` と同じ |
| `\D` | `[^0-9]` と同じ |
| `\s` | `[ \t\n\r\f\v]` と同じ |
| `\S` | `[^ \t\n\r\f\v]` と同じ |
| `\w` | `[a-zA-Z0-9_]` と同じ |
| `\W` | `[^a-zA-Z0-9_]` と同じ |

マッチの基本原則：

  * 第 1 原則: **最初のマッチが優先される（最左一致）**
  * 第 2 原則: **繰り返し制御文字は可能な限り長いマッチを行う（最長一致）**

オンラインの正規表現チェックツール:

  * [Debuggex](https://www.debuggex.com/)
  * [regex101.com](https://regex101.com/)

### re

Python は言語レベルで正規表現をサポートしていない。正規表現の機能は標準ライブラリの `re` モジュールにより提供される。

正規表現は、Python の文字列リテラルで記述する。このとき、先に文字列リテラルとしての解釈が行われ、その結果に対して正規表現としての解釈が行われることに注意する。raw-string でない限り、エスケープシーケンス `'\\'` については二つの解釈が競合しているので、正規表現 `'\\'` を表すには文字列リテラルを `'\\\\'` としなければならない（文字列リテラルとして `'\\'` を `'\'` と解釈するからである）。これが煩わしいので、正規表現は普通 raw-string で記述する。

正規表現を使う方法は 2 通りある。

  1. `re.compile(pattern, flags=0)` 関数で正規表現 `pattern` を明示的にコンパイルし、返される正規表現オブジェクトのメソッドを使う。
  2. `re` モジュールが提供する関数を使う（正規表現のコンパイルは暗黙的に行われる）。

`re` モジュールが提供する関数に共通するオプション `flags` には、次の定数を指定することができる。

| 定数名 | 意味 |
|:--:|:---|
| `re.A` または `re.ASCII` | `\D`、`\s`、`\S`、`\w`、`\W` が ASCII 文字限定でマッチする |
| `re.I` または `re.IGNORECASE` | 大文字・小文字を区別しないでマッチする |
| `re.M` または `re.MULTILINE` | 複数行のテキストを対象としたときに `^` と `$` が各行の先頭と末尾にマッチする |
| `re.S` または `re.DOTALL` | `.` が改行にもマッチする |

たとえば、次のコード

``` python
prog = re.compile(pattern)
result = prog.search(string)
```

は、以下と同等となる。

``` python
result = re.search(pattern, string)
```

1 の方法、すなわち、正規表現オブジェクトのメソッドを使う場合は、共通するオプション `pos`, `endpos` で検索範囲を制限できる（指定できないメソッドもある）。`pos` を指定した場合はその位置から文字列を走査するが、省略した場合は文字列の最初から走査する。`endpos` を指定した場合はその位置より後ろの走査を禁止するが、省略した場合は文字列の末尾まで走査すること許可する。

2 の方法、すなわち、`re` モジュールが提供する関数で探索する場合は、常に対象の文字列の先頭から走査し、末尾まで走査することを許可する。

正規表現の構文上の問題がある場合は、`re.error` 例外が発生する。例外インスタンスには、次の追加属性がある。

| 属性 | 意味 |
|:---|:---|
| `msg` | フォーマットされていないエラーメッセージ |
| `pattern` | 正規表現のパターン |
| `pos` | `pattern` のコンパイルに失敗した場所のインデックス（`None` の場合もある） |
| `lineno` | `pos` に対応する行（`None` の場合もある） |
| `colno` | `pos` に対応する列（`None` の場合もある） |

In [ ]:
import re
try:
    re.search(r"a(a", "aa")  # パターンでカッコが閉じていない
except re.error as err:
    print(f"{err.msg = }")
    print(f"{err.pattern = }")
    print(f"{err.pos = }")

err.msg = 'missing ), unterminated subpattern'
err.pattern = 'a(a'
err.pos = 1


``` python
Pattern.search(string[, pos[, endpos]])
re.search(pattern, string, flags=0)
```

これらは、文字列 `string` を走査して、正規表現がどこにマッチするか調べる。マッチしていればマッチオブジェクトを返し、そうでなければ `None` を返す。

マッチオブジェクトのメソッドは次のとおり:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `Match.group([group1, ...])` | 正規表現にマッチした文字列を返す | `str` or `tuple` |
| `Match.groups(default=None)` | パターンにマッチした全てのサブグループについてマッチした文字列を含むタプルを返す。`default` 引<br /><br />数はマッチする文字列が存在しない場合に返す値を指定する | `tuple` |
| `Match.groupdict(default=None)` | パターンにマッチした全てのサブグループについてサブグループ名をキーとする辞書を返す。`default` <br /><br />引数はマッチする文字列が存在しない場合に返す値を指定する | `dict` |
| `Match.expand(template)` | テンプレート文字列に対して `\1` または `\g<name>` の形式でサブグループを指定すると、マッチした文<br /><br />字列に置き換えられる | `str` |
| `Match.start([group])` | マッチの開始位置を返す | `int` |
| `Match.end([group])` | マッチの終了位置を返す。終了位置は最後の文字の後ろの位置であることに注意 | `int` |
| `Match.span([group])` | マッチの位置 (start, end) を含むタプルを返す | `tuple` |

`Match.group()` メソッドは、引数なしで正規表現全体にマッチした文字列を返すが、正規表現を `()` でグルーピングしている場合にサブグループの番号を引数として渡されると、そのサブグループについてマッチした文字列を返す。全体が番号 0 であり（したがって `m.group()` と `m.group(0)` は同じ）、サブグループは左から右へ 1, 2, ... と番号付けされる。サブグループの番号を複数渡すこともでき、その場合、`Match.group()` はそれらのグループに対応する結果を含むタプルを返す。サブグループが正規表現のマッチしなかった部分に含まれているなら、対応する結果は `None` となる。サブグループが正規表現の複数回マッチした部分に含まれているなら、最後のマッチが返される。

実のところ、マッチオブジェクトは `__getitem__()` をサポートしているので、インデックスでサブグループにアクセスできる。すなわち、`m.group(0)`, `m.group(1)`, `m.group(2)` はそれぞれ `m[0]`, `m[1]`, `m[2]` と同等である。

`Match.groups()` メソッドは `Match.group(1,2,3,...)` と等価であり、全てのサブグループを含むタプルを返す。

`Match.start()`、`Match.end()`、`Match.span()` にも、サブグループの番号を 1 つ引数として渡すことができる。この場合、これらの関数は、そのサブグループについての結果を返す。

In [ ]:
import re

text = "spam ham eggs"

m = re.search(r"am", text)
if m is not None:
    assert m.span() == (2, 4)  # 最左一致原則により spam の中の am にマッチ
else:
    print("no match")

m = re.search(r"gs|gg|eg", text)
if m is not None:
    assert m.group() == "eg"  # 最左一致原則により eg でマッチ
else:
    print("no match")

m = re.search(r"(\d+)?(\d?).+(am)(.+)", text)
if m is not None:
    assert m.group(1) is None  # サブグループ (\d+) は正規表現のマッチしなかった部分に含まれている
    assert m.group(2) == ""  # サブグループ (\d?) は空文字列にマッチしている
    assert m.span(3) == (6, 8) and m.group(4) == " eggs"  # サブグループ (am) は2回マッチしているが、最後のマッチが返される
    assert m.group(1, 2, 3, 4) == (m[1], m[2], m[3], m[4]) == m.groups()  # インデックスでもアクセスできる
    assert m.expand(r"HIT:\4") == "HIT: eggs"  # テンプレート文字列を使用
else:
    print("no match")

m = re.search(r"(?P<sub1>\d+)?(?P<sub2>\d?).+(?P<sub3>am)(?P<sub4>.+)", text)
if m is not None:
    assert m.groupdict() == {'sub1': None, 'sub2': '', 'sub3': 'am', 'sub4': ' eggs'}
    assert m.expand(r"HIT:\g<sub4>") == "HIT: eggs"  # テンプレート文字列にサブグループ名を指定
else:
    print("no match")

``` python
Pattern.match(string[, pos[, endpos]])
re.match(pattern, string, flags=0)
```

文字列 `string` の先頭（または `pos`）から正規表現にマッチした場合に限りマッチオブジェクトを返し、そうでなければ `None` を返す。

In [ ]:
import re

line = "Subject: Re: happy birthday"
m = re.match(r"Subject: (Re: )?(.*)", line)
if m is not None:
    assert m.group(1, 2) == (m[1], m[2]) == m.groups() == ("Re: ", "happy birthday")
else:
    print("no match")

pattern = re.compile(r"re: (.*)", re.I)
assert pattern.match(line) is None  # pattern は文字列の先頭からではマッチしない
m = pattern.match(line, 9, 18)
if m is not None:
    assert m[1] == "happy"
else:
    print("no match")

``` python
Pattern.fullmatch(string[, pos[, endpos]])
re.fullmatch(pattern, string, flags=0)
```

文字列 `string` 全体（または `pos` 以降、または `pos` から `endpos` までの部分）が正規表現にマッチした場合に限りマッチオブジェクトを返し、そうでなければ `None` を返す。

In [ ]:
import re

s = 'aaa@xxx.com'
m = re.fullmatch(r'[a-z]+@[a-z]+\.com', s)
if m is not None:
    assert m[0] == s
else:
    print("no match")

``` python
Pattern.findall(string[, pos[, endpos]])
re.findall(pattern, string, flags=0)
```

文字列 `string` 中の正規表現にマッチした文字列全てをリストで返す。

In [ ]:
import re

text = "abcdeあいうえお"
assert re.findall(r'\w', text, re.A) == ['a', 'b', 'c', 'd', 'e']

m_text = '''0110010
1010101
0111110
'''
assert re.findall(r'^0[01]+0$', m_text, re.M) == ['0110010', '0111110']

``` python
Pattern.finditer(string[, pos[, endpos]])
re.finditer(pattern, string, flags=0)
```

文字列 `string` において、正規表現の重複しない全てのマッチに対して、マッチオブジェクトを 1 つずつ生成するイテレーターを返す。文字列は左から右に走査され、見つかった順序でマッチオブジェクトが yield される。空の一致が結果に含まれる。

In [ ]:
import re

text = "spam ham eggs"
for m in re.finditer(r"\S+am", text):
    print(m[0])

spam
ham


``` python
Pattern.split(string, maxsplit=0)
re.split(pattern, string, maxsplit=0, flags=0)
```

文字列 `string` を正規表現にマッチした文字列で分割した結果のリストを返す。`maxsplit` が 0 でなければ、分割は最大 `maxsplit` 回起こり、残りの文字列はリストの最後の要素として返される。

In [ ]:
import re
re.split(r'\W+', 'Words, words, words.')  # 文末の . 後ろの空文字列にもマッチする

['Words', 'words', 'words', '']

``` python
Pattern.sub(repl, string, count=0)
re.sub(pattern, repl, string, count=0, flags=0)
```

文字列 `string` 中で正規表現にマッチした部分列を `repl` に置き換えた文字列を返す。マッチしなかった場合、`string` がそのまま返される。オプション引数 `count` が与えられている場合、先頭から `count` 個の マッチ部分列だけを置換する。正規表現を `()` でグルーピングしているときは、`repl` 中の `\番号` でサブグループにマッチした部分列を参照できる。これを後方参照と呼ぶ。`repl` が raw-string でない場合は、後方参照が `\\番号` の形となる。

In [ ]:
import re

text = "fuga@mail.com"
assert re.sub(r"(\w+)@(\w+)", r"\1.reply@\2", text) == "fuga.reply@mail.com"

### 注意点

正規表現の処理については、マッチする場合の処理だけでなく、マッチしない場合の処理にも注意が必要である。なぜならば、アルゴリズムの性質上、マッチしない場合の最悪のケースでの処理が、マッチする場合の処理よりはるかに時間がかかるからである。とくに、この傾向は、精度が低い正規表現の処理では顕著になる。ここで、正規表現の**精度**が低いとは、正規表現が `'*'` や `'+'` を含むため必要以上に多くの文字列にマッチすることをいう。精度が低い正規表現の処理では、マッチしない文字列に対して、マッチしていないものと確認されるまでに何度も試行され、計算量が増える。

たとえば、次の 2 つの正規表現を考える。

  1. `.* (.*)\[(.*)\]:.*`
  2. `[12]\d{3}-[01]\d-[0-3]\d ([^ \[]*?)\[([^\]]*?)\]:.*`

1 の正規表現は、`'*'` を 4 つも含み、明確な文字は角括弧 `[]` とコロン `:` しかないため、2 の正規表現より多くの文字列にマッチして精度が低い。どちらの正規表現でも、以下の文字列変数 `good_input` を与えた場合に、`'app'` と `'web.1'` を抽出することができる。

In [ ]:
import re

good_input = '2014-08-26 app[web.1]: 50.0.134.125 - - [26/Aug/2014 00:27:41] "GET / HTTP/1.1" 200 14 0.0005'
m1 = re.match(r".* (.*)\[(.*)\]:.*", good_input)
m2 = re.match(r"[12]\d{3}-[01]\d-[0-3]\d ([^ \[]*?)\[([^\]]*?)\]:.*", good_input)
if m1 is not None and m2 is not None:
    assert m1.groups() == m2.groups() == ("app", "web.1")
else:
    print("no match")

しかしながら、以下のマッチしない文字列 `bad_input` に対して、1 の正規表現の処理は 2 の正規表現の処理の 10 倍も時間がかかる。

In [ ]:
import re

bad_input = '50.0.134.125 - - [26/Aug/2014 00:27:41] "GET / HTTP/1.1" 200 14 0.0005'
%timeit -n 1000 re.match(r".* (.*)\[(.*)\]:.*", bad_input)
%timeit -n 1000 re.match(r"[12]\d{3}-[01]\d-[0-3]\d ([^ \[]*?)\[([^\]]*?)\]:.*", bad_input)

24 µs ± 5.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


精度が低い正規表現である種のものは、処理時間の爆発を引き起こす。これはサーバー攻撃に悪用できるので、セキュリティ上の弱点となる（この攻撃は ReDoS と呼ばれる）。

たとえば、正規表現 `'^(\w+)+$'` はこの種の危険な正規表現の例となる。実際、意地悪く末尾にだけマッチしない文字がある文字列を与える場合、文字列の長さを 10 文字から 1 文字増やすごとに処理時間が 約 2 倍に増える。

In [ ]:
import re

pattern = re.compile(r"^(\w+)+$")
%timeit -n 100 -r 5 pattern.match("ABCabc012@")
%timeit -n 100 -r 5 pattern.match("ABCabc0123@")
%timeit -n 100 -r 5 pattern.match("ABCabc01234@")
%timeit -n 100 -r 5 pattern.match("ABCabc012345@")

67.6 µs ± 5.91 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)
154 µs ± 16.6 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)
681 µs ± 124 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)
664 µs ± 216 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)


危険な正規表現は以下のツールである程度は検証可能であるが、正規表現についてユーザーからの入力文字列をそのまま埋め込んで使うことは避けるべきである。

  * [ReDoS checker](https://devina.io/redos-checker)

一般に、正規表現では、明確な文字・文字クラスを使用し、`{n}`, `{m, n}` で繰り返し回数を指定する場合に精度が高くなるが、やりすぎると可読性が低くなって、バグが入りやすい（実際、上記の 2 の正規表現が何を抽出するのか一目見て理解するのは困難である）。

1 つの正規表現でいろいろなことを解決しようとはせず、if 文や文字列メソッドなどと正規表現との組み合わせを考えるようにする。そうすれば、マッチしない文字列をできるだけ排除した上で（精度が低い）正規表現で処理するという方法がとれる。

Unicode データベース
--------------------

[Unicode Character Database（UCD）](http://www.unicode.org/Public/UCD/latest/ucd/) は、 Unicode の文字に関するデータを集めたデータベースである。

標準ライブラリの `unicodedata` モジュールは、UCD へのアクセスを提供する。

### コードポイントの名前

``` python
unicodedata.lookup(name)
```

この関数は、Unicode 規格でコードポイントに付けられた名前（文字列）を受け取り、対応する文字を返す。`name` という名前が定義されていない場合には、`KeyError` 例外を発生させる。

``` python
unicodedata.name(chr[, default])
```

この関数は、第 1 引数に 1 文字の文字列を受け取り、Unicode 規格でそのコードポイントに付けられた名前（文字列）を返す。名前が定義されていない場合には第 2 引数を返すが、第 2 引数が与えられていなければ `ValueError` 例外を発生させる。

In [ ]:
import unicodedata
assert unicodedata.lookup("Sushi") == "🍣"
assert unicodedata.name("A") == "LATIN CAPITAL LETTER A"
assert "\N{LATIN CAPITAL LETTER A}" == "A"
for c in "👨‍👩‍👧‍👦":
    name = unicodedata.name(c)
    print(f"{name}: {unicodedata.lookup(name)}")

MAN: 👨
ZERO WIDTH JOINER: ‍
WOMAN: 👩
ZERO WIDTH JOINER: ‍
GIRL: 👧
ZERO WIDTH JOINER: ‍
BOY: 👦


### 結合クラス

``` python
unicodedata.combining(chr)
```

この関数は、第 1 引数に 1 文字の文字列を受け取り、Unicode 規格の中でその文字に割り当てられた Canonical_Combining_Class 属性の値を返す。

Canonical_Combining_Class 属性の値は、結合クラスを示すもので、結合文字の並び順を決定するために使われる。

  * 0 → 非結合文字（通常の文字）
  * 0 以外 → 結合文字（発音記号、濁点、半濁点など）

In [ ]:
import unicodedata
for c in "ééがが":  # 見た目が同じだが単一のコードポイントで結合文字を使用していないものと、結合文字を使用しているものを並べている
    print(f"{c}: {unicodedata.combining(c)}")

é: 0
e: 0
́: 230
が: 0
か: 0
゙: 8


### 全角・半角

``` python
unicodedata.east_asian_width(chr)
```

この関数は、1 文字の文字列を受け取り、Unicode 規格の中でその文字に割り当てられた East Asian Width 属性の値を返す。

East_Asian_Width 属性の値は文字の画面上の横幅（文字幅）に関するヒントを与えるもので、`F`（Fullwidth）、`H`（Halfwidth）、`W`（Wide）、`Na`（Narrow）、`A`（Ambiguous）、`N`（Neutral）という 6 種類からなる（画像は [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:East_Asian_Width_1.svg) より）。

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/East_Asian_Width_1.svg/443px-East_Asian_Width_1.svg.png)

日本語環境の場合、文字幅は

  * `F`、`W` なら全角（= 2 桁の幅）
  * `H`、`Na` なら半角（= 1 桁の幅）

として扱えばよい。

属性値 `A` は、まさに曖昧（Ambiguous）で、日本語環境でも、文字幅の扱いが一貫していないことがある。例えば属性値 `A` は `■` のような全角記号に付与されていたり、発音記号付きのラテン文字など、最近の Unicode に対応するフォントでは半角扱いされる文字にも付与されている。

属性値 `N` が付与される文字は、日本語環境で一般に使われるものではない。

In [ ]:
import unicodedata
assert unicodedata.east_asian_width('Ａ') == 'F'  # 全角英数
assert unicodedata.east_asian_width('ｱ') == 'H'   # 半角カナ
assert unicodedata.east_asian_width('あ') == 'W'  # 全角かな
assert unicodedata.east_asian_width('a') == 'Na'  # 半角英数
assert unicodedata.east_asian_width('α') == 'A'  # ギリシア文字
assert unicodedata.east_asian_width('■') == 'A'  # 全角記号
assert unicodedata.east_asian_width('é') == 'A'   # 発音記号付きラテン文字

次のコードは、文字列に対して、コードポイント数ではなく文字幅の合計を返す関数の例である。この関数は、結合文字やゼロ幅ジョインナーにも対応している。East_Asian_Width 属性値 `A` を全角とみなしているので、発音記号付きのラテン文字などが含まれる場合、環境によっては文字幅を正しく扱えないことに注意する。

In [ ]:
import unicodedata


def text_width(text):
    """
    Calculate character width of text
    """
    count = 0
    prev_width = 0

    for c in text:
        # zero width joiner
        if ord(c) == 8205:
            count -= prev_width

        # canonical combining class
        elif unicodedata.combining(c) > 0:
            pass

        # Zenkaku
        elif unicodedata.east_asian_width(c) in "FWA":
            prev_width = 2
            count += 2

        # Hankaku
        else:
            prev_width = 1
            count += 1

    return count


if __name__ == "__main__":
    assert text_width("abＡＢ■") == 8
    # 合成絵文字にも対応する
    text1 = "👨‍👩‍👧‍👦"
    assert len(text1) == 7
    assert text_width(text1) == 2
    # 結合文字にも対応する
    text2 = "がが"  # 左の文字は単一のコードの「が」で、右の文字は結合文字を使用
    assert len(text2) == 3
    assert text_width(text2) == 4
    assert text_width("é") == 2  # 発音記号付きラテン文字などには対応しないことがある

### Unicode 正規化

Unicode 文字を、等価性と呼ばれるものに基づいて統一的な内部表現に変換する操作を **Unicode 正規化**という。Unicode には 2 種類の等価性がある。

  * **正準等価性**: 見た目も意味的にも区別できない文字  
たとえば、濁音・半濁音のひらがな・カタカナについて、単一のコードポイントによって表現される文字と、仮名+濁点・半濁点の 2 つのコードポイントを使った結合文字によって表現される文字との等価性。
  * **互換等価性**: 見た目が異なり意味的にも異なるかもしれない文字  
たとえば、半角と全角、普通の数字と丸数字・上付き・下付きは、それぞれ互換文字とされる。

Unicode 正規化には、正準等価性と互換等価性という基準と、単一のコードポイントへの置き換えと複数のコードポイントへの分解という基準により、4種類の形式が定義されている。

| | 置き換え | 分解 |
|:--:|:--:|:--:|
| 正準等価性 | NFC | NFD |
| 互換等価性 | NFKC | NFKD |

``` python
unicodedata.normalize(form, unistr)
```

この関数は、Unicode 文字列 `unistr` を、`form` で指定した正規化形式により正規化した文字列を返す。

In [ ]:
import unicodedata

s = "１２３ａｂｃｱｲｳｴｵ①②③¹²³"
# NFC、NFD では変換されない
assert unicodedata.normalize("NFC", s) == unicodedata.normalize("NFD", s) == s
# NFKC、NFKD では変換される。NFKD でも単一のコードポイントへの置き換えとなる
assert unicodedata.normalize("NFKC", s) == unicodedata.normalize("NFKD", s) == "123abcアイウエオ123123"

s = "がガぱパ"
# NFC、NFKC では変換されない
assert unicodedata.normalize("NFC", s) == unicodedata.normalize("NFKC", s) == s
# NFD、NFKD では、見た目は同じ文字列を返す
assert (x1 := unicodedata.normalize("NFD", s)) == (x2 := unicodedata.normalize("NFKD", s)) == "がガぱパ"
# 実際には、NFD、NFKD では結合文字に変換している
assert len(x1) == len(x2) == 8  # 4 文字なのに文字数が 8 とされる
assert list(x1) == list(x2) == ["か", "゙", "カ", "゙", "は", "゚", "ハ", "゚"]

Unicode 正規化を使うことによって、半角全角などの表記の揺れを統一して検索などを行うことができる。この場合、正規化形式は、単一のコードポイントへの置き換えとなる NFKC を使うとよい。

In [ ]:
import re

text = "これはＰｙｔｈｏｎというプログラミング言語で書かれています"
n_text = unicodedata.normalize("NFKC", text)
m = re.search(r"python", n_text, re.I)
if m is not None:
    assert m.start() == 3
else:
    print("no match")

テキストの整形
--------------

標準ライブラリの `textwrap` モジュールは、一定の文字数でテキストを折り返したり、切り詰めたりして整形するための `textwrap.TextWrapper` クラスを提供する。

`textwrap.TextWrapper` のコンストラクタ引数は、同名の属性を設定する。属性とデフォルトの値は次のとおり。

| 属性 | 意味 | デフォルト |
|:---|:---|:---|
| `width` | 折り返し（改行）が行われる行の最大の長さ | `70` |
| `initial_indent` | 最初の行の先頭に挿入される文字列。最初の行の長さに加算される | `''` |
| `subsequent_indent` | 2 行目以降の行の先頭に挿入される文字列。それらの行の長さに加算される | `''` |
| `expand_tabs` | `True` の場合、タブ文字が空白に展開される | `True` |
| `replace_whitespace` | `True` の場合、（`expand_tabs` が `True` ならタブの展開の後に）各種空白文字（'\t\n\v\f\r'）をそれぞれスペース 1 文字に<br /><br />置換する | `True` |
| `fix_sentence_endings` | `True` の場合、英文テキストの文の終わりを検出し、文の区切りを 2 つの空白で区切る | `False` |
| `break_long_words` | `True` の場合、`width` より長い語は切られる | `True` |
| `drop_whitespace` | `True` の場合、インデント処理の前に各行の最初と最後の空白文字を削除する。削除される空白文字が行全体に及ぶ場合<br /><br />は、行自体を削除する | `True` |
| `break_on_hyphens` | `True` の場合、英語で一般的なように、折り返しは空白か合成語に含まれるハイフンの直後で行われる | `True` |
| `tabsize` | `expand_tabs` が `True` の場合、タブ文字は `tabsize` を埋めるようにスペースに展開される | `8` |
| `placeholder` | 切り詰める場合に出力の最後の行に置く文字列。コンストラクタ引数はキーワード専用であることに注意する | `' [...]'`|
| `max_lines` | `None` 以外の場合、出力は行数 `max_lines` を超えないようにされ、切り詰める際には出力の最後の行を `placeholder`<br /><br /> に置き換える。コンストラクタ引数はキーワード専用であることに注意する | `None` |

`textwrap.TextWrapper` インストタンスのメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `wrap(text)` | `text` を、すべての行が高々 `width` 文字の長さになるような文字列のリストに変換して返す | `list` |
| `fill(text)` | `"\n".join(self.wrap(text))` を返す | `str` |

これらのメソッドを 1 回だけ使用するなら、簡単に呼び出せる次の関数を使用するとよい。

``` python
textwrap.wrap(text, width=70, **kwargs)
textwrap.fill(text, width=70, **kwargs)
textwrap.shorten(text, width, **kwargs)
```

これらの関数の定義で docstring を省略したものは、次のようになっている。

``` python
def wrap(text, width=70, **kwargs):
    w = TextWrapper(width=width, **kwargs)
    return w.wrap(text)

def fill(text, width=70, **kwargs):
    w = TextWrapper(width=width, **kwargs)
    return w.fill(text)

def shorten(text, width, **kwargs):
    w = TextWrapper(width=width, max_lines=1, **kwargs)
    return w.fill(' '.join(text.strip().split()))
```

簡易関数を何度も呼び出す場合、呼び出すたびに `TextWrapper` インストタンスを作り直すことになってコストが高くなる。この場合は、`TextWrapper` インストタンスを作成しておき、メソッドを呼び出すほうが良い。インストタンスの属性値を変更することで `wrap()` メソッドの振る舞いを変更することが可能である。

In [ ]:
import textwrap

text = """Python is a programming language that lets you work more quickly and integrate your systems more effectively.
You can learn to use Python and see almost immediate gains in productivity and lower maintenance costs."""

# テキストの折り返し
t1 = textwrap.fill(text)
print(t1)

print("---")

# テキストの切り詰め
t2 = textwrap.fill(text, max_lines=2)
print(t2)

print("---")

# テキストの折り畳み
t3 = textwrap.shorten(text, 20)
print(t3)

# インストタンスメソッドを直接使用
w = textwrap.TextWrapper()
assert w.fill(text) == t1
w.max_lines = 2
assert w.fill(text) == t2
w.width = 20
w.max_lines = 1
assert w.fill(text) == t3

Python is a programming language that lets you work more quickly and
integrate your systems more effectively. You can learn to use Python
and see almost immediate gains in productivity and lower maintenance
costs.
---
Python is a programming language that lets you work more quickly and
integrate your systems more effectively. You can learn to use [...]
---
Python is a [...]


`TextWrapper` クラスの `wrap()` メソッドは、`len()` 関数を使用して文字数（正確には Unicode コードポイント数）で折り返し位置を決めるため、全角半角が混じったテキストでは画面上の横幅が統一されない。

この問題を解決するためには、`TextWrapper` クラスのサブクラスを作成し、もとのメソッドの中で使用されている `len()` 関数を、全て「文字幅を計算する関数」（`text_width()` のコードを参照）に書き換えたメソッドでオーバーライドする必要がある。